In [ ]:
# import

import hashlib
import json
from time import time
import random
import requests
from flask import Flask, request, jsonify

In [ ]:
class Blockchain(object):
    def __init__(self):
        # __init__ 블록체인 객체를 생성한다.
        # 객체의 구성 요소로는 블록들이 저장되는 체인과 블록 내에 저장될 거래 내역 리스트 current_transaction,
        # 그리고 블록체인을 운영하는 노드들의 정보인nodes,
        # 마지막으로 블록체인 첫 생성 시 자동으로 첫 블록(genesis block)을 생성하는 코드(new_block)로 구성된다.
        self.chain = []
        self.current_transaction = []
        self.nodes = set()
        self.new_block(previous_hash=1, proof=100)

    # 해시암호화 함수
    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    #마지막 블록 호출 함수
    @property
    def last_block(self):
        return self.chain[-1]
    
    #블록 검증 함수
    @staticmethod
    def valid_proof(last_proof, proof):
        guess = str(last_proof + proof).encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == '0000'
    
    #작업 증명(pow) 함수
    def pow(self, last_proof):
        proof = random.randint(-1000000, 1000000)
        while self.valid_proof(last_proof, proof) is False:
            proof = random.randint(-1000000, 1000000)
        return proof
    
    #거래 내역 추가 함수
    def new_transaction(self, sender, recipient, amount):
        self.current_transaction.append(
            {
                'sender' : sender, #송신자
                'recipient' : recipient, #수신자
                'amount' : amount, #금액
                'timestamp':time()
            }
        )
        return self.last_block['index'] + 1
    
    #신규 블록 생성 함수
    def new_block(self, proof, previous_hash = None):
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time(), #timestamp from 1970
            'transactions' : self.current_transaction,
            'nonce' : proof,
            'previous_hash' : previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transaction = []
        self.chain.append(block)
        return block
    
    #블록 검증 함수
    def valid_chain(self, chain):
        last_block = chain[0]
        current_index = 1


        while current_index < len(chain):
            block = chain[current_index]
            print('%s' % last_block)
            print('%s' % block)
            print("\n-----------\n")
            if block['previous_hash'] != self.hash(last_block):
                return False
            last_block = block
            current_index += 1
        return True

In [ ]:
blockchain = Blockchain()
my_ip = '0.0.0.0'
my_port = '5000'
node_identifier = 'node_'+my_port
mine_owner = 'master'
mine_profit = 0.1



app = Flask(__name__)



#블록 정보 호출 함수
@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain,
        'length' : len(blockchain.chain),
    }
    return jsonify(response), 200


#신규 거래 추가 함수
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount']

    if not all(k in values for k in required):
        return 'missing values', 400
    

    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])
    response = {'message' : 'Transaction will be added to Block {%s}' % index}

    return jsonify(response), 201


#채굴 함수
@app.route('/mine', methods=['GET'])
def mine():
    print("MINING STARTED")
    last_block = blockchain.last_block
    last_proof = last_block['nonce']
    proof = blockchain.pow(last_proof)
    blockchain.new_transaction(
        sender=mine_owner,
        recipient=node_identifier,
        amount=mine_profit #coinbase transaction
    )


    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    print("MINING FINISHED")


    response = {
        'message' : 'new block found',
        'index' : block['index'],
        'transactions' : block['transactions'],
        'nonce' : block['nonce'],
        'previous_hash' : block['previous_hash']
    }


    return jsonify(response), 200


#노드의 시작
app.run()